<a href="https://colab.research.google.com/github/Sadhamh/Redbus/blob/main/redbus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd

class RedbusScraper:
    def __init__(self, driver_path):
        options = Options()
        options.headless = True
        service = Service(driver_path)
        self.driver = webdriver.Chrome(service=service, options=options)

    def scrape_bus_data(self, url):
        self.driver.get(url)
        time.sleep(5)

        bus_data = []
        buses = self.driver.find_elements(By.CLASS_NAME, 'bus-class-name')

        for bus in buses:
            route_name = bus.find_element(By.CLASS_NAME, 'route-name-class').text
            bus_name = bus.find_element(By.CLASS_NAME, 'bus-name-class').text
            bus_type = bus.find_element(By.CLASS_NAME, 'bus-type-class').text
            departing_time = bus.find_element(By.CLASS_NAME, 'departing-time-class').text
            duration = bus.find_element(By.CLASS_NAME, 'duration-class').text
            reaching_time = bus.find_element(By.CLASS_NAME, 'reaching-time-class').text
            star_rating = float(bus.find_element(By.CLASS_NAME, 'star-rating-class').text)
            price = float(bus.find_element(By.CLASS_NAME, 'price-class').text.replace('₹', ''))
            seats_available = int(bus.find_element(By.CLASS_NAME, 'seats-available-class').text)

            bus_data.append([
                route_name, '', bus_name, bus_type, departing_time,
                duration, reaching_time, star_rating, price, seats_available
            ])

        self.driver.quit()

        return pd.DataFrame(bus_data, columns=[
            'route_name', 'route_link', 'busname', 'bustype', 'departing_time',
            'duration', 'reaching_time', 'star_rating', 'price', 'seats_available'
        ])


In [ ]:
import mysql.connector
from mysql.connector import Error

class DatabaseManager:
    def __init__(self, host, user, password, database):
        self.host = host
        self.user = user
        self.password = password
        self.database = database

    def create_connection(self):
        try:
            connection = mysql.connector.connect(
                host=self.host,
                user=self.user,
                password=self.password,
                database=self.database
            )
            return connection
        except Error as e:
            print(f"Error: '{e}'")
            return None

    def create_table(self):
        connection = self.create_connection()
        cursor = connection.cursor()
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS bus_routes (
            id INT AUTO_INCREMENT PRIMARY KEY,
            route_name TEXT,
            route_link TEXT,
            busname TEXT,
            bustype TEXT,
            departing_time DATETIME,
            duration TEXT,
            reaching_time DATETIME,
            star_rating FLOAT,
            price DECIMAL(10, 2),
            seats_available INT
        )
        """)
        connection.commit()
        cursor.close()
        connection.close()

    def insert_data(self, df):
        connection = self.create_connection()
        cursor = connection.cursor()

        for _, row in df.iterrows():
            cursor.execute("""
            INSERT INTO bus_routes (route_name, route_link, busname, bustype, departing_time,
            duration, reaching_time, star_rating, price, seats_available)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, (
                row['route_name'], row['route_link'], row['busname'], row['bustype'], row['departing_time'],
                row['duration'], row['reaching_time'], row['star_rating'], row['price'], row['seats_available']
            ))

        connection.commit()
        cursor.close()
        connection.close()


In [ ]:
import streamlit as st
import pandas as pd
from database_manager import DatabaseManager
def fetch_data_from_db():
    db_manager = DatabaseManager(host='localhost', user='your_username', password='your_password', database='redbus')
    connection = db_manager.create_connection()
    query = "SELECT * FROM bus_routes"
    df = pd.read_sql(query, connection)
    connection.close()
    return df


df = fetch_data_from_db()


st.title("Redbus Data Filtering Application")

bus_type = st.selectbox('Bus Type', df['bustype'].unique())
route_name = st.selectbox('Route Name', df['route_name'].unique())
price_range = st.slider('Price Range', float(df['price'].min()), float(df['price'].max()))

filtered_data = df[
    (df['bustype'] == bus_type) &
    (df['route_name'] == route_name) &
    (df['price'] >= price_range[0]) &
    (df['price'] <= price_range[1])
]

st.write(filtered_data)
